# Brusselator model

The dynamics of the oscillating reaction discovered by Belousov and Zhabotinsky, 
can be modeled through the so-called Brusselator model depending on two parameters:

$$
\left\{\begin{aligned}
\frac{dy_1}{dt} & = 1 - (b+1) y_1 + a y_1^2y_2\\
\frac{dy_2}{dt} & = b y_1 - a y_1^2y_2
\end{aligned}\right.
$$

In [1]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter

from ipywidgets import interact, IntSlider, Dropdown

from mylib.model import brusselator_model
import mylib.integration as integration

output_notebook(hide_banner=True)

## Quasi-exact solution

The quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 5 with stepsize control and fine tolerances due to Dormand and Prince.

In [2]:
def plot_quasi_exact_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.line(sol.t, sol.y[0], legend="y1", line_width=2)    
    fig_sol.line(sol.t, sol.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    show(fig_sol)
    
plot_quasi_exact_sol()

## Characterisation of stiffness

In [3]:
def plot_eigenvalue():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5 , 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)
    
    eig_vals = np.zeros((sol.t.size, 2), dtype=np.complex_)
    for it in range(0,sol.t.size):
        eig_vals[it], _ = np.linalg.eig(jac(0, sol.y[:,it]))

    lambda1 = eig_vals[:, 0]
    lambda2 = eig_vals[:, 1]
    
    fig_real = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                      title = "Real part of eigenvalues (click on legend entries to hide corresponding plot)")
    fig_imag = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                      title = "Imaginary part of eigenvalues (click on lengend to hide corresponding plot)")
            
    fig_real.line(sol.t, np.real(lambda1), line_width=2, legend="lambda1")
    fig_real.line(sol.t, np.real(lambda2), line_width=2, color="Green", legend="lambda2")
    fig_real.legend.click_policy="hide"

    fig_imag.line(sol.t, np.imag(lambda1), line_width=2, legend="lambda1")
    fig_imag.line(sol.t, np.imag(lambda2), line_width=2, color="Green", legend="lambda2")
    fig_imag.legend.click_policy="hide"

    show(column(fig_real, fig_imag))
    
plot_eigenvalue()

## Runge-Kutta methods

In [4]:
def plot_rk_methods():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0.
    tend = 20.
    
    yini = (1.5 , 3)
    
    nt = 1000
    sol_rk = integration.forward_euler(tini, tend, nt, yini, fcn)
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk.y[1])
    
    # plot exact and rk solutions 
    fig_sol = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Solution")
    plt_sol_y1 = fig_sol.x(sol_rk.t, sol_rk.y[0], line_width=2, legend="y1")
    plt_exa_y1 = fig_sol.line(sol_rk.t, sol_exa.y[0], color="Grey")
    plt_sol_y2 = fig_sol.x(sol_rk.t, sol_rk.y[0], line_width=2, color="Green", legend="y2")
    plt_exa_y1 = fig_sol.line(sol_rk.t, sol_exa.y[1], color="Grey")
    fig_sol.legend.location = "top_left"

    # plot global error
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=900, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    plt_err_y1 = fig_err.x(sol_rk.t, y1_err, line_width=2, legend="y1")
    plt_err_y2 = fig_err.x(sol_rk.t, y2_err, line_width=2, color="Green", legend="y2")
    fig_err.legend.location = "top_left"

    show(column(fig_sol, fig_err), notebook_handle=True)
    
    def update(method, nt):
        t = np.linspace(tini, tend, nt)
        if method == "rk1":
            sol_rk = integration.forward_euler(tini, tend, nt, yini, fcn)
        elif method == "rk2":
            sol_rk = integration.rk2(tini, tend, nt, yini, fcn)
        elif method == "rk3":
            sol_rk = integration.rk3(tini, tend, nt, yini, fcn)
        elif method == "rk4":
            sol_rk = integration.rk4(tini, tend, nt, yini, fcn)        

        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk.t)
        y1_err = np.abs(sol_exa.y[0] - sol_rk.y[0])
        y2_err = np.abs(sol_exa.y[1] - sol_rk.y[1])
        
        plt_sol_y1.data_source.data = dict(x=sol_rk.t, y=sol_rk.y[0])
        plt_sol_y2.data_source.data = dict(x=sol_rk.t, y=sol_rk.y[1])
        plt_err_y1.data_source.data = dict(x=sol_rk.t, y=y1_err)
        plt_err_y2.data_source.data = dict(x=sol_rk.t, y=y2_err)
        push_notebook()
        
    interact(update,
             nt=IntSlider(min=100, max=5000, value=nt, step=100, continuous_update=False),
             method=Dropdown(options=["rk1", "rk2", "rk3", "rk4"], value='rk1', description='Method:'))

plot_rk_methods()

interactive(children=(Dropdown(description='Method:', options=('rk1', 'rk2', 'rk3', 'rk4'), value='rk1'), IntS…

### Computational cost

In [5]:
def plot_rk_cost():

    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5 , 3)
    
    l_nt = [101, 1001, 10001, 100001]
    fe_rk1=[]
    norm_rk1=[]
    fe_rk2=[]
    norm_rk2=[]
    fe_rk3=[]
    norm_rk3=[]
    fe_rk4=[]
    norm_rk4=[]
    
    for nt in l_nt:
        
        t = np.linspace(tini,tend,nt)
        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=t)
       
        sol_rk1 = integration.forward_euler(tini, tend, nt, yini, fcn)
        fe_rk1.append(sol_rk1.nfev)
        norm_rk1.append(np.linalg.norm(sol_exa.y-sol_rk1.y) / np.sqrt(nt))

        sol_rk2 = integration.rk2(tini, tend, nt, yini, fcn)
        fe_rk2.append(sol_rk2.nfev)
        norm_rk2.append(np.linalg.norm(sol_exa.y-sol_rk2.y) / np.sqrt(nt))
        
        sol_rk3 = integration.rk3(tini, tend, nt, yini, fcn)
        fe_rk3.append(sol_rk3.nfev)
        norm_rk3.append(np.linalg.norm(sol_exa.y-sol_rk3.y) / np.sqrt(nt))
        
        if (nt == 100001):
            nt_rk4 = 20001
            sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", 
                                rtol=1.e-12, atol=1.e-12, t_eval=np.linspace(tini,tend,nt_rk4))
            sol_rk4 = integration.rk4(tini, tend, nt_rk4, yini, fcn)
            fe_rk4.append(4*(nt_rk4-1))
            norm_rk4.append(np.linalg.norm(sol_exa.y-sol_rk4.y) / np.sqrt(nt_rk4))
        else:
            sol_rk4 = integration.rk4(tini, tend, nt, yini, fcn)
            fe_rk4.append(sol_rk4.nfev)
            norm_rk4.append(np.linalg.norm(sol_exa.y-sol_rk4.y) / np.sqrt(nt))

        
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=400, plot_width=950,
                 title= "Number of function evaluations vs norm of global error" )
    
    fig.x(norm_rk1, fe_rk1, legend="rk1")
    fig.line(norm_rk1, fe_rk1, legend="rk1")
    fig.x(norm_rk2, fe_rk2, legend="rk2", color="Green")
    fig.line(norm_rk2, fe_rk2, legend="rk2", color="Green")
    fig.x(norm_rk3, fe_rk3, legend="rk3", color="Red")
    fig.line(norm_rk3, fe_rk3, legend="rk3", color="Red")
    fig.x(norm_rk4, fe_rk4, legend="rk4", color="Indigo")
    fig.line(norm_rk4, fe_rk4, legend="rk4", color="Indigo")
    show(fig)
    
    
plot_rk_cost()

### Implicit Euler

In [6]:
def plot_implicit_euler_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    nt = 20
    dt = (tend-tini)/(nt-1)
    
    yini = (1.5 , 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)
 
    sol_be = integration.backward_euler(tini, tend, nt, yini, fcn)
    
    sol_eig = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_be.t)
    fcn_stab = np.zeros((sol_be.t.size, 2))
    real_eig_vals = np.zeros((sol_be.t.size))
    for it in range(0,sol_be.t.size):
        eig_val, _ = np.linalg.eig(jac(0., sol_eig.y[:,it]))
        real_eig_vals[it] = np.real(eig_val)[0]
        fcn_stab[it] = np.absolute(1-dt*eig_val)
        
    mask = np.where(real_eig_vals>0)
    fcn_stab_1 = fcn_stab[:,0]
    fcn_stab_2 = fcn_stab[:,0]

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.line(sol.t, sol.y[0], color="black", line_dash="dotted")    
    fig_sol.line(sol.t, sol.y[1], color="black", line_dash="dotted")    
    fig_sol.x(sol_be.t, sol_be.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_be.t, sol_be.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_stab = figure(x_range=(tini, tend), width=950, height=300, title="Value of | 1 - z |")
    fig_stab.line(sol_be.t, np.ones(sol_be.t.size), color="black", line_dash="dotted")
    fig_stab.x(sol_be.t, fcn_stab_1, line_width=2, legend='Value of  | 1-z | for negative eigen values')
    fig_stab.x(sol_be.t, fcn_stab_2, line_width=2)
    fig_stab.x(sol_be.t[mask], fcn_stab_1[mask], line_width=2, color="crimson", legend='Value of  | 1-z | for positive eigen values')    
    fig_stab.x(sol_be.t[mask], fcn_stab_2[mask], line_width=2, color="crimson")
    fig_stab.legend.background_fill_alpha = 0.5
        
    show(column(fig_sol, fig_stab))

plot_implicit_euler_sol()

## Enbedded Runge-Kutta methods

### Method based on 3/8 fourth-order Runge-Kutta method

In [33]:
def plot_rk_embedded_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    nt = 200
    
    yini = (1.5 , 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

    tol = 1.e-4
    sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
    glob_err = np.empty(sol_rk_emb.t.size)
    for i in range(sol_rk_emb.t.size):
        glob_err[i] = np.sqrt(1./2) * np.linalg.norm(sol_exa.y[:,i] - sol_rk_emb.y[:,i])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.line(sol.t, sol.y[0], color="Grey")
    fig_sol.line(sol.t, sol.y[1], color="Grey")
    plt_y1 = fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[0], legend="y1", line_width=2)    
    plt_y2 = fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
  
    fig_dt = figure(x_range=(tini, tend), plot_height=300, plot_width=950, y_axis_type="log", 
                    title = "Accepted (black) and rejected (red) times step") 
    plt_dt = fig_dt.square(sol_rk_emb.t[0:-1], sol_rk_emb.dt, color="Black")
    plt_line_dt = fig_dt.line(sol_rk_emb.t[0:-1], sol_rk_emb.dt, color="Black")
    plt_dt_rej = fig_dt.circle_x(sol_rk_emb.t_rej, sol_rk_emb.dt_rej, color="Crimson")

    fig_err = figure(x_range=(tini, tend),y_axis_type="log", width=700, height=300, title="Error")
    plt_loc_err = fig_err.x(sol_rk_emb.t, sol_rk_emb.loc_err_est, legend="estimate local error", line_width=2)
    plt_loc_err_exa = fig_err.circle(sol_rk_emb.t, sol_rk_emb.loc_err, legend="local error", fill_alpha=0., color="crimson")
    plt_glob_err = fig_err.square(sol_rk_emb.t, glob_err, legend="global error", fill_alpha=0, color="green")
    plt_tol = fig_err.line(sol_rk_emb.t, tol*np.ones(sol_rk_emb.t.size), color="grey", line_dash="dotted")    
    fig_err.legend.location = "top_left"
    fig_err.legend.orientation = "horizontal"

    show(column(fig_sol, fig_dt, fig_err), notebook_handle=True)
    
    def update(tol):
        
        sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
        
        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
        glob_err = np.empty(sol_rk_emb.t.size)
        for i in range(sol_rk_emb.t.size):
            glob_err[i] = np.sqrt(1./2) * np.linalg.norm(sol_exa.y[:,i] - sol_rk_emb.y[:,i])


        plt_y1.data_source.data = dict(x=sol_rk_emb.t, y=sol_rk_emb.y[0])
        plt_y2.data_source.data = dict(x=sol_rk_emb.t, y=sol_rk_emb.y[1]) 

        plt_dt.data_source.data = dict(x=sol_rk_emb.t[0:-1], y=sol_rk_emb.dt)
        plt_line_dt.data_source.data = dict(x=sol_rk_emb.t[0:-1], y=sol_rk_emb.dt) 
        plt_dt_rej.data_source.data = dict(x=sol_rk_emb.t_rej, y=sol_rk_emb.dt_rej)
        
        plt_loc_err.data_source.data = dict(x=sol_rk_emb.t, y=sol_rk_emb.loc_err_est)
        plt_loc_err_exa.data_source.data  = dict(x=sol_rk_emb.t, y=sol_rk_emb.loc_err)
        plt_glob_err.data_source.data = dict(x=sol_rk_emb.t, y=glob_err)
        plt_tol.data_source.data = dict(x=sol_rk_emb.t, y=tol*np.ones(sol_rk_emb.t.size))

        print("   Total : "+str(len(sol_rk_emb.t[0:-1])))
        print("   Rejected times : "+str(len(sol_rk_emb.t_rej)))
        print("   Rejected rates : ", len(sol_rk_emb.t_rej)/len(sol_rk_emb.t[0:-1]))
        print("   Number of time step : " + str(sol_rk_emb.t.size-1))
        print("   Number of function evaluations : " + str(sol_rk_emb.nfev))
        print((sol_rk_emb.nfev/(sol_rk_emb.t.size-1)-4)/4)
        push_notebook()
        
        
    dtol={'1.e-1':1.e-1, '1.e-2':1.e-2, '1.e-3':1.e-3, '1.e-4':1.e-4, '1.e-5':1.e-5, '1.e-6':1.e-6}     
    interact(update, tol=Dropdown(options=dtol, value=1.e-4, description='tol'))


plot_rk_embedded_sol()

interactive(children=(Dropdown(description='tol', index=3, options={'1.e-1': 0.1, '1.e-2': 0.01, '1.e-3': 0.00…

### Computational cost

In [30]:
import warnings
warnings.filterwarnings('ignore')

def plot_embedded_cost():

    yini = (1.5 , 3.)
    tini = 0.
    tend = 20.
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
            
    l_nt = [101, 501, 1001, 5001, 10001, 20001]
    fe_rk38=[]
    norm_rk38=[]
    
    for nt in l_nt:
        sol_exa = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=np.linspace(tini, tend, nt))
        sol_yrk38 = integration.rk38(tini, tend, nt, yini, fcn)
        fe_rk38.append(sol_yrk38.nfev)
        #fe_rk38.append(sol_yrk38.t.size-1)
        norm_rk38.append(np.linalg.norm(sol_exa.y-sol_yrk38.y) / np.sqrt(nt))
        
    l_tol = [1.e-2, 1.e-4, 1.e-6, 1.e-8, 1.e-10, 1.e-12]
    fe_rk_emb=[]
    norm_rk_emb=[]
    
    for tol in l_tol:
        sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
        sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
        #sol_rk_emb.t.size-1
        fe_rk_emb.append(sol_rk_emb.nfev)
        #fe_rk_emb.append(sol_rk_emb.t.size-1)
        err = 0 
        for i in range(sol_rk_emb.dt.size):
            ldt = sol_rk_emb.dt[i]
            err += ldt * ((sol.y[0,i+1]-sol_rk_emb.y[0,i+1])*(sol.y[0,i+1]-sol_rk_emb.y[0,i+1]) + \
                           (sol.y[1,i+1]-sol_rk_emb.y[1,i+1])*(sol.y[1,i+1]-sol_rk_emb.y[1,i+1])  )
        err = np.sqrt(err)    
        norm_rk_emb.append(err)
    
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, title= "Number of function evaluations vs norm of global error"
                )
    #fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, title= "Number of time steps vs norm of global error" )
    fig.x(norm_rk38, fe_rk38, legend="rk38", color="Indigo")
    fig.line(norm_rk38, fe_rk38, legend="rk38", color="Indigo")
    fig.x(norm_rk_emb, fe_rk_emb, legend="rk embedded", color="Green")
    fig.line(norm_rk_emb, fe_rk_emb, legend="rk embedded", color="Green")

    show(fig)
    
plot_embedded_cost()

## Dormand and Price method

In [31]:
def plot_dopri5_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12)

    fig_sol = figure(x_range=(tini, tend),  plot_height=300, plot_width=900, title="Solution")
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=900, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")

    tol = 1.e-4
    sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_dopri5.t)
       
    y1_err = np.abs(sol_exa.y[0] - sol_dopri5.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_dopri5.y[1])

    fig_sol.line(sol.t, sol.y[0], color="Grey")
    fig_sol.line(sol.t, sol.y[1], color="Grey")
    plt_sol_y1 = fig_sol.x(sol_dopri5.t, sol_dopri5.y[0], legend="y1", line_width=2)    
    plt_sol_y2 = fig_sol.x(sol_dopri5.t, sol_dopri5.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    plt_err_y1 = fig_err.x(sol_dopri5.t, y1_err, line_width=2, legend="y1")
    plt_err_y2 = fig_err.x(sol_dopri5.t, y2_err, line_width=2, color="Green", legend="y2")
    
    show(column(fig_sol, fig_err), notebook_handle=True)
    
    def update(tol):
        
        sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)
        sol_exa = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_dopri5.t)
        
        y1_err = np.abs(sol_exa.y[0] - sol_dopri5.y[0])
        y2_err = np.abs(sol_exa.y[1] - sol_dopri5.y[1])

        plt_sol_y1.data_source.data = dict(x=sol_dopri5.t, y=sol_dopri5.y[0])
        plt_sol_y2.data_source.data = dict(x=sol_dopri5.t, y=sol_dopri5.y[1]) 
        plt_err_y1.data_source.data = dict(x=sol_dopri5.t, y=y1_err)
        plt_err_y2.data_source.data = dict(x=sol_dopri5.t, y=y2_err) 

        print("   Number of time step : " + str(sol_dopri5.t.size))
        print("   Number of function evaluations : " + str(sol_dopri5.nfev))
        
        push_notebook()
        
    dtol={'1.e-2':1.e-2, '1.e-4':1.e-4, '1.e-6':1.e-6, '1.e-8':1.e-8, '1.e-10':1.e-10}     
    interact(update, tol=Dropdown(options=dtol, value=1.e-4, description='tol'))

plot_dopri5_sol()

interactive(children=(Dropdown(description='tol', index=1, options={'1.e-2': 0.01, '1.e-4': 0.0001, '1.e-6': 1…

## time steps and evaluation time, time cost

In [39]:
import warnings
warnings.filterwarnings('ignore')

def plot_embedded_cost():

    yini = (1.5 , 3.)
    tini = 0.
    tend = 20.
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
            
    l_nt = [101, 501, 1001, 5001, 10001, 20001]
    
    '''
    fe_rk38=[]
    norm_rk38=[]
    for nt in l_nt:
        sol_exa = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=np.linspace(tini, tend, nt))
        sol_yrk38 = integration.rk38(tini, tend, nt, yini, fcn)
        fe_rk38.append(sol_yrk38.nfev)
        #fe_rk38.append(sol_yrk38.t.size-1)
        norm_rk38.append(np.linalg.norm(sol_exa.y-sol_yrk38.y) / np.sqrt(nt))
    '''   
    l_tol = [1.e-2, 1.e-4, 1.e-6, 1.e-8, 1.e-9,1.e-10, 1.e-11,1.e-12]
    fe_rk_emb=[]
    norm_rk_emb=[]

    fe_dopri=[]
    norm_dopri=[]

    
    for tol in l_tol:
        sol_dopri = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)
        sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-13, atol=1.e-13, t_eval=sol_dopri.t)
        #sol_rk_emb.t.size-1
        #fe_dopri.append(sol_dopri.nfev)
        fe_dopri.append(sol_dopri.t.size-1)
        err = 0 
        #return sol_dopri
        for i in range(sol_dopri.t.size):
            ldt = sol_dopri.t[i]
            err += ldt * ((sol.y[0,i]-sol_dopri.y[0,i])*(sol.y[0,i]-sol_dopri.y[0,i]) + \
                           (sol.y[1,i]-sol_dopri.y[1,i])*(sol.y[1,i]-sol_dopri.y[1,i])  )
        err = np.sqrt(err)    
        norm_dopri.append(err)
    
    for tol in l_tol:
        sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
        sol = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-13, atol=1.e-13, t_eval=sol_rk_emb.t)
        #sol_rk_emb.t.size-1
        #fe_rk_emb.append(sol_rk_emb.nfev)
        fe_rk_emb.append(sol_rk_emb.t.size-1)
        err = 0 
        for i in range(sol_rk_emb.dt.size):
            ldt = sol_rk_emb.dt[i]
            err += ldt * ((sol.y[0,i+1]-sol_rk_emb.y[0,i+1])*(sol.y[0,i+1]-sol_rk_emb.y[0,i+1]) + \
                           (sol.y[1,i+1]-sol_rk_emb.y[1,i+1])*(sol.y[1,i+1]-sol_rk_emb.y[1,i+1])  )
        err = np.sqrt(err)    
        norm_rk_emb.append(err)
    
    #fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, title= "Number of function evaluations vs norm of global error"
    #           )
    #fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, title= "Number of time steps vs norm of global error" )
    #fig.x(norm_dopri, fe_dopri, legend="DOPRI", color="Indigo")
    #fig.line(norm_dopri, fe_dopri, legend="DOPRI", color="Indigo")
    #fig.x(norm_rk_emb, fe_rk_emb, legend="rk embedded", color="Green")
    #fig.line(norm_rk_emb, fe_rk_emb, legend="rk embedded", color="Green")
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900,
                 title= "accepted time steps vs tolerance")
    fig.x(l_tol, fe_dopri, legend="DOPRI", color="Indigo")
    fig.line(l_tol, fe_dopri, legend="DOPRI", color="Indigo")
    fig.x(l_tol, fe_rk_emb, legend="rk embedded", color="Green")
    fig.line(l_tol, fe_rk_emb, legend="rk embedded", color="Green")
    fig.legend.location = "top_left"
    show(fig)
    
a=plot_embedded_cost()